In [34]:
import pandas as pd
import numpy as np
from itertools import combinations
from functools import reduce
from collections import Iterable

# use for vertify 
from mlxtend.frequent_patterns import apriori, association_rules

In [35]:
# df = pd.read_csv(
#     '/home/sokhorn/sokhorn/dataSet/data/sample_data_set.csv', sep=',')
# df.drop(['Unnamed: 0'], axis=1, inplace=True)
# df
df = pd.read_csv(
    '/home/sokhorn/sokhorn/dataSet/data/Online Retail.csv', sep=',')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

/home/sokhorn/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,InvoiceNo,StockCode,lower,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,white metal lantern,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,knitted union flag hot water bottle,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,red woolly hottie white heart.,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...,...
541919,581598,22613,NaN,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541920,581599,22899,NaN,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541921,581600,23254,NaN,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541922,581601,23255,NaN,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [36]:
sample_dataset = df[[
    'InvoiceNo',
    'StockCode',
    'Quantity'
]]
sample_dataset


,InvoiceNo,StockCode,Quantity
0,536365,85123A,6
1,536365,71053,6
2,536365,84406B,8
3,536365,84029G,6
4,536365,84029E,6
...,...,...,...
541919,581598,22613,12
541920,581599,22899,6
541921,581600,23254,4
541922,581601,23255,4


In [37]:
item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
item_sets


StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C581490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C581499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
def en_code(x):
    if x > 0:
        return 1
    else:
        return 0

In [39]:
item_sets = item_sets.applymap(en_code)
item_sets

StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
from itertools import combinations

In [41]:
cc = list(combinations(item_sets.columns, 1))
cc

[('10002',),
 ('10080',),
 ('10120',),
 ('10123C',),
 ('10123G',),
 ('10124A',),
 ('10124G',),
 ('10125',),
 ('10133',),
 ('10134',),
 ('10135',),
 ('11001',),
 ('15030',),
 ('15034',),
 ('15036',),
 ('15039',),
 ('15044A',),
 ('15044B',),
 ('15044C',),
 ('15044D',),
 ('15056BL',),
 ('15056N',),
 ('15056P',),
 ('15056bl',),
 ('15056n',),
 ('15056p',),
 ('15058A',),
 ('15058B',),
 ('15058C',),
 ('15060B',),
 ('15060b',),
 ('16008',),
 ('16010',),
 ('16011',),
 ('16012',),
 ('16014',),
 ('16015',),
 ('16016',),
 ('16020C',),
 ('16033',),
 ('16043',),
 ('16045',),
 ('16046',),
 ('16048',),
 ('16049',),
 ('16052',),
 ('16053',),
 ('16054',),
 ('16151A',),
 ('16156L',),
 ('16156S',),
 ('16161G',),
 ('16161M',),
 ('16161P',),
 ('16161U',),
 ('16162L',),
 ('16162M',),
 ('16168M',),
 ('16169E',),
 ('16169K',),
 ('16169M',),
 ('16169N',),
 ('16169P',),
 ('16202A',),
 ('16202B',),
 ('16202E',),
 ('16206B',),
 ('16207A',),
 ('16207B',),
 ('16216',),
 ('16218',),
 ('16219',),
 ('16225',),
 ('16235

In [42]:
item_sets[sorted(cc[0])].sum() / len(item_sets)


StockCode
10002    0.002778
dtype: float64

In [43]:
item_sets


StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
k_itemst = item_sets[['22748', '22749']]
k_itemst.loc[(k_itemst['22748'] > 0) & (k_itemst['22749'] > 0)]


StockCode,22748,22749
InvoiceNo,,
536367,1,1
536592,1,1
537050,1,1
537468,1,1
538104,1,1
...,...,...
580367,1,1
580399,1,1
580730,1,1


In [45]:
item_sets[['22748', '22749']]


StockCode,22748,22749
InvoiceNo,,
536365,0,0
536366,0,0
536367,1,1
536368,0,0
536369,0,0
...,...,...
C581484,0,0
C581490,0,0
C581499,0,0


In [46]:
item_sets[['22748', '22749']].values


array([[0, 0],
       [0, 0],
       [1, 1],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [47]:
support_count = 0
for i in item_sets[['22748', '22749']].values:
    if(np.all(i != 0)):
        support_count += 1
support_count


74

In [48]:
item_sets


StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
item_sets[sorted(cc[1])].values[1] = 0

In [50]:
def calculate_support(k_itemst, trnaction_size=len(item_sets.index)):
    support_count = 0
    for i in k_itemst.values:
        if(np.all(i != 0)):
            support_count += 1
    return support_count / trnaction_size

**_Frequent Itemset Generation_**


In [51]:
# min_support = 0.2
# k = 1
# item_sest_1 = combinations(item_sets.columns, k)
# c1 = [
#     {
#         "Item": x,
#         "Support":  calculate_support(item_sets[sorted(x)])
#     }
#     for x in sorted(item_sest_1)
#     if calculate_support(item_sets[sorted(x)]) >= min_support
# ]
# frequent_kitemset = c1
# while(True):

#     k += 1
#     item_sest_k = combinations(item_sets.columns, k)

#     c_k = [
#         {
#             "Item": x,
#             "Support":  calculate_support(item_sets[sorted(x)])
#         }
#         for x in sorted(item_sest_k)
#         if calculate_support(item_sets[sorted(x)]) >= min_support
#     ] # generate untill we found ck equal to null 

#     if(len(c_k) == 0):
#         break
#     else:
#         frequent_kitemset.append(c_k)


# flat_list = []
# for sublist in frequent_kitemset:
#     if(len(sublist) == 2):
#         flat_list.append(sublist)
#     else:
#         for item in sublist:
#             flat_list.append(item)
# fk = pd.DataFrame(flat_list)
#fk

### old code  

**Other Function of Apriori**


In [53]:
# def APRIORI_MY(data, min_support=0.04,  max_length=4):
#     # Collecting Required Library
#     import numpy as np
#     import pandas as pd
#     from itertools import combinations
#     # Step 1:
#     # Creating a dictionary to stored support of an itemset.
#     support = {}
#     L = list(data.columns)

#     # Step 2:
#     # generating combination of items with len i in ith iteration
#     for i in range(1, max_length+1):
#         c = set(combinations(L, i))

#     # Reset "L" for next ith iteration
#         L = set()
#     # Step 3:
#         # iterate through each item in "c"
#         for j in list(c):

#             sup = data.loc[:, j].product(axis=1).sum()/len(data.index)

#             if sup > min_support:
#                 #print(sup, j)
#                 support[j] = sup

#                 # Appending frequent itemset in list "L", already reset list "L"
#                 L = list(set(L) | set(j))

#     # Step 4: data frame with cols "items", 'support'
#     result = pd.DataFrame(list(support.items()), columns=["Items", "Support"])
#     return(result)



# min_support = 0.2
# item_sest_1 = combinations(item_sets.columns, 1)
# c1 = [
#     {
#         "Item": x,
#         "Support":  calculate_support(item_sets[sorted(x)])
#     }
#     for x in sorted(item_sest_1)
#     if calculate_support(item_sets[sorted(x)]) >= min_support
# ]

# k = 1
# frequent_kitemset = c1
# while(len(frequent_kitemset) != 0):

#     k += 1
#     item_sest_k = combinations(item_sets.columns, k)

#     c_k = [
#         {
#             "Item": x,
#             "Support":  calculate_support(item_sets[sorted(x)])
#         }
#         for x in sorted(item_sest_k)
#         if calculate_support(item_sets[sorted(x)]) >= min_support
#     ]

#     if(len(c_k) == 0):
#         break
#     else:
#         frequent_kitemset.append(c_k)


# flat_list = []
# for sublist in frequent_kitemset:
#     if(len(sublist) == 2):
#         flat_list.append(sublist)
#     else:
#         for item in sublist:
#             flat_list.append(item)
# fk = pd.DataFrame(flat_list)


In [54]:
# fk = APRIORI_MY(item_sets)
# fk

**Apriori**

In [55]:
item_sets

StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
item_name =  item_sets.columns
item_name

Index(['10002', '10080', '10120', '10123C', '10123G', '10124A', '10124G',
       '10125', '10133', '10134',
       ...
       'M', 'PADS', 'POST', 'S', 'gift_0001_10', 'gift_0001_20',
       'gift_0001_30', 'gift_0001_40', 'gift_0001_50', 'm'],
      dtype='object', name='StockCode', length=4070)

In [57]:
items =  item_sets.to_numpy()
items

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

*Generating Itemset*

In [58]:
# generint first itemset 
C = {}
L = {}
Discard = {}
itemset_size = 1
Discard.update({itemset_size : []})
C.update({itemset_size : [ [f] for f in item_name ]})

In [59]:
item_sets

StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Generating RC Column**

In [60]:
item_sets['RC'] = 0
item_sets

StockCode,10002,10080,10120,10123C,10123G,10124A,10124G,10125,10133,10134,...,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m,RC
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C581484,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C581499,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
item_np =  item_sets.values
item_np

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [62]:
# dup = []
# for i in range(len(item_np)):
#     item_i = item_np[i]
#     RC = 1
#     dup_list = []
#     for j in range(len(item_np)):
#         item_j = item_np[j]
#         if(i != j):
#             print(f'{i} : {item_i}, {j} : {item_j}')
#             if (item_i == item_j).all():
#                 RC += 1
#                 dup.append({
#                     item_i: RC
#                 })
#     for d in dup:
#         if (item_i != d):
#             dup.append({
#                 item_i: RC
#             })
#     print("\n")
# dup


In [63]:
# unique_rows, index = np.unique(item_np, axis=0)
# print(unique_rows)

we want to update RC Only with item itemset in row of tranction 
We increase only RC when item are duplicate 

In [64]:
item_np

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [65]:
item_rc = []
for i in range(len(item_np)):
    item_i = item_np[i]
    RC = 1
    aa = {
        "Item": item_i,
        "RC": RC
    }
    item_rc.append(aa)
    for j in range(len(item_np)):
        item_j = item_np[j]
        if i != j:
            if (item_i == item_j).all():
                RC += 1
                item_rc[i] = {
                    "Item": item_i,
                    "RC": RC
                }
item_rc


In [ ]:
df_rc =  pd.DataFrame(item_rc)
df_rc

In [ ]:
df_rc['Item'] = df_rc['Item'].apply(lambda x : str(x))
df_rc.drop_duplicates(inplace=True)

In [ ]:
def clean_str(str):
    str_list =  list((str.replace("[","").replace("]","").split(" ")))
    return [int (i) for i in str_list]

In [ ]:
df_rc['Item'] =  df_rc['Item'].apply(clean_str)
split_df = pd.DataFrame(df_rc['Item'].tolist(), columns=item_sets.columns)

In [ ]:
item_sets

In [ ]:
split_df.set_index(df_rc.index, inplace=True)
split_df['RC'] = df_rc['RC']
split_df

In [ ]:
RC = split_df['RC'].values
RC

In [ ]:
aa = split_df[['21730', '22752', '71053', '84029E', '84029G']].values
aa

In [ ]:
def support_k_itemst(k_itemst):
    s = 0
    for i in range(len(k_itemst)):
        s += reduce(lambda a, b: a & b, k_itemst[i] & RC[i])
    return s

In [ ]:
C = {}
L = {}
Discard = {}
itemset_size = 1

In [ ]:
def count_ocurence(itemset, Tranctions):
    count = 0
    for i in range(len(Tranctions)):
        if set(itemset).issubset(set(Tranctions[i])):
            count += 1
    return count

In [ ]:
def join_set_item(set_of_its, order_column_name):
    C = []
    for i in range(len(set_of_its)):
        for j in range(i + 1, len(set_of_its)):
            it_out = join_two_itemsets(
                set_of_its[i], set_of_its[j], order_column_name)
            if(len(it_out)) > 0:
                C.append(it_out)

    return C


In [ ]:
def join_two_itemsets(it1, it2, order):
    it1.sort(key=lambda x : order.index(x))
    it2.sort(key=lambda x : order.index(x))

    for i in range(len(it1) - 1): # check befor the last one 
       if it1[i] != it2 [i] :
           return []
    
    if order.index(it1[-1]) < order.index(it2[-1]) :
        return [it1] + [it2[-1]]

    return []


In [ ]:
def mergeKItemIntoOne(ab):
    result = []
    if(len(ab) != 0):
        if(len(ab[0]) == 1):
            return ab
        else:
            for item in ab:
                res = item[0] + item[1].split()
                result.append(res)
        return result


In [ ]:
def get_frequent(itemesets, min_support, prev_discard):
    L = []
    support_count = []
    new_discard = []
    column_items = mergeKItemIntoOne(itemesets)
   
    k = len(prev_discard)
    for i in range(len(itemesets)):
        discard_before = False
        item = itemesets[i] 
        result = []
        for i_item in item:
            if isinstance(i_item, list):
                for j in i_item:
                    result.append(j)
            else:
                result.append(i_item)

        if k > 0:
            for it in prev_discard[k]:
                if set(it).issubset(set(result)):
                    discard_before = True
                    break
        
        if not discard_before:
            count = support_k_itemst(split_df[column_items[i]].values)
            if count >= min_support:
                L.append(result)
                support_count.append(count)
            else:
                new_discard.append(result)
    return L, support_count, new_discard

In [ ]:
def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

**Generating 1 itemse base on support count of row**

In [ ]:
C = {}
L = {}
Discard = {}
itemset_size = 1
Discard.update({itemset_size : []})

In [ ]:
combine = combinations(split_df.loc[:, split_df.columns != "RC"].columns, 1)
itemset_size = 1
min_support = 0.5
itemsets = []
for item in combine:
    coll_name = item[0].replace("[","").replace("]","").replace(",","")
    supp = support_k_itemst(split_df[[coll_name]].values)
    if supp >= min_support:
        itemsets.append([coll_name])
C.update({itemset_size : itemsets})

In [ ]:
support_count = {}
f, supp, new_discard = get_frequent(
    C[itemset_size], min_support, Discard
)
Discard.update({itemset_size: new_discard})
L.update({itemset_size: f})
support_count.update({itemset_size: supp})

**To Be Continue**

In [ ]:
order = list(split_df.loc[:, split_df.columns != 'RC'].columns)

In [ ]:
k = itemset_size + 1
convergence = False
while not convergence:
    C.update({k: join_set_item(L[k - 1], order)})
    f, supp, new_discard = get_frequent(C[k], min_support, Discard)
    L.update({k: f})
    Discard.update({k: new_discard})
    support_count.update({k: supp})
    if(len(L[k]) == 0):
        convergence = True
    k += 1

===========================**Vertify with lib**=======================================

In [ ]:
itemset = apriori(split_df.loc[:,split_df.columns != "RC"], min_support=min_support, use_colnames=True)
itemset

**Apriori Algorithms**